In [1]:
#https://blog.naver.com/dev-dog

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
#파일 넣기기
input_file = "한국_기업문화_HR_데이터셋_샘플.csv"
#1. 파일 확인: 
df = pd.read_csv(input_file)
df

,Unnamed: 0,Age,이직여부,출장빈도,일일성과지표,부서,집까지거리,학력수준,전공분야,EmployeeCount,...,대인관계만족도,StandardHours,스톡옵션등급,총경력,연간교육횟수,워라밸,현회사근속년수,현직무근속년수,최근승진후경과년수,현상사근속년수
0,1041,28,No,Travel_Rarely,866,Sales,5,3,Medical,1,...,4,80,0,6,4,3,5,4,1,3
1,184,53,No,Travel_Rarely,1084,Research & Development,13,2,Medical,1,...,3,80,2,5,3,3,4,2,1,3
2,1222,24,Yes,Travel_Rarely,240,Human Resources,22,1,Human Resources,1,...,3,80,1,1,2,3,1,0,0,0
3,67,45,No,Travel_Rarely,1339,Research & Development,7,3,Life Sciences,1,...,3,80,1,25,2,3,1,0,0,0
4,220,36,No,Travel_Rarely,1396,Research & Development,5,2,Life Sciences,1,...,4,80,0,16,3,4,13,11,3,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1293,43,No,Non-Travel,343,Research & Development,9,3,Life Sciences,1,...,3,80,0,7,2,2,3,2,1,2
996,1324,29,No,Travel_Rarely,726,Research & Development,29,1,Life Sciences,1,...,4,80,2,11,3,3,7,0,1,6
997,977,34,No,Non-Travel,999,Research & Development,26,1,Technical Degree,1,...,3,80,3,5,2,3,5,4,0,0
998,547,42,Yes,Travel_Frequently,933,Research & Development,19,3,Medical,1,...,4,80,0,7,2,3,2,2,2,2


In [ ]:
'''
2. 데이터 전처리 (15점)

결측치 처리 여부를 확인하고 필요한 전처리를 수행하시오. 

이직여부를 이진값(0, 1)으로 변환하시오.

나머지 범주형 피처는 인코딩하시오.
'''
missing=(df.isnull().sum()/ len(df)*100).sort_values(ascending=False)# 결측치 없음음
missing

df['이직여부'].value_counts()
for i in df['이직여부'].index:
    if df['이직여부'][i] == 'Yes':
        df['이직여부'][i]=1
    else:
        df['이직여부'][i]=0
df['이직여부'].value_counts() #target label set
    
# 변수 분류 (OneHot & gender or Label)
label_encode_cols = []
onehot_cols = []

for col in df.columns:
    if df[col].dtype == 'object' and col not in ['Gender', 'gender', 'Sex', 'sex']:
        n_unique = df[col].nunique()
        if n_unique > 2:
            label_encode_cols.append(col)
        elif n_unique == 2:
            onehot_cols.append(col)

# 범주형 변수 인코딩 함수 (LabelEncoder 사용, 실패 시 해시 기반)
def encode_categoricals(df):
    cat_cols = df.select_dtypes(include=['object', 'category']).columns
    cat_cols = [col for col in cat_cols if col != 'year']
    for col in cat_cols:
        try:
            df[col] = LabelEncoder().fit_transform(df[col])
        except:
            df[col] = df[col].astype(str).apply(lambda x: hash(x) % 1000)  # 해시값으로 대체
    return df

label_encode_cols
df[label_encode_cols]=encode_categoricals(df[label_encode_cols]) # Label

df #one-hot은 차원 커지니깐 보류 했음.


C:\Users\kimye\AppData\Local\Temp\ipykernel_11152\3088058348.py:18: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df['이직여부'][i]=0
C:\Users\kimye\AppData\Local\Temp\ipykernel_11152\3088058348.py:18: SettingWithCopyWarning: 
A value is trying 

,Unnamed: 0,Age,이직여부,출장빈도,일일성과지표,부서,집까지거리,학력수준,전공분야,EmployeeCount,...,대인관계만족도,StandardHours,스톡옵션등급,총경력,연간교육횟수,워라밸,현회사근속년수,현직무근속년수,최근승진후경과년수,현상사근속년수
0,1041,28,0,2,866,2,5,3,3,1,...,4,80,0,6,4,3,5,4,1,3
1,184,53,0,2,1084,1,13,2,3,1,...,3,80,2,5,3,3,4,2,1,3
2,1222,24,0,2,240,0,22,1,0,1,...,3,80,1,1,2,3,1,0,0,0
3,67,45,0,2,1339,1,7,3,1,1,...,3,80,1,25,2,3,1,0,0,0
4,220,36,0,2,1396,1,5,2,1,1,...,4,80,0,16,3,4,13,11,3,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1293,43,0,0,343,1,9,3,1,1,...,3,80,0,7,2,2,3,2,1,2
996,1324,29,0,2,726,1,29,1,1,1,...,4,80,2,11,3,3,7,0,1,6
997,977,34,0,0,999,1,26,1,5,1,...,3,80,3,5,2,3,5,4,0,0
998,547,42,0,1,933,1,19,3,3,1,...,4,80,0,7,2,3,2,2,2,2


In [ ]:
'''
3. 
피처 선택 (15점)

이직 여부 예측에 유의미하다고 생각되는 피처 5~10개을 선택하고, 선택 근거를 간략하게 기술하시오.

피처 선정에 관한 자세한 기술은 이후에 다룰 예정.(현재는 예측력보다 이해/해석 중점)
feature_target=['이직여부', '업무만족도', '부서', '근무환경만족도', '직무', '연봉인상률', '총경력',
       '야근여부', '성과등급', '워라밸', '현회사근속년수']
'''

df.columns

#업무 만족도, '워라밸, '근무환경만족도': 높아야 이직 안함
#'연봉인상률': 낮으면 이직 하고 돈 안 들어오니 나가고 싶음
#'야근여부': 많으면 나가고 싶음.

feature_target=['이직여부', '업무만족도', '워라밸', '연봉인상률', '총경력','야근여부','근무환경만족도']


Index(['Unnamed: 0', 'Age', '이직여부', '출장빈도', '일일성과지표', '부서', '집까지거리', '학력수준',
       '전공분야', 'EmployeeCount', 'EmployeeNumber', '근무환경만족도', '성별', '시간당급여',
       '업무몰입도', '직급', '직무', '업무만족도', '결혼상태', '월급여', 'MonthlyRate', '이전회사경험수',
       'Over18', '야근여부', '연봉인상률', '성과등급', '대인관계만족도', 'StandardHours', '스톡옵션등급',
       '총경력', '연간교육횟수', '워라밸', '현회사근속년수', '현직무근속년수', '최근승진후경과년수', '현상사근속년수'],
      dtype='object')